In [3]:
!pip install matplotlib torch scipy librosa cn2an opencc

In [ ]:
pip install torch==2.0.0+cu117 -f https://download.pytorch.org/whl/torch_stable.html

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

hps = utils.get_hparams_from_file("./models/config.json")

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("./models/321_epochs.pth", net_g, None)

DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
INFO:root:Loaded checkpoint './models/321_epochs.pth' (iteration None)


In [15]:
!cp ./jyutjyu.json ./venv/lib/site-packages/opencc/clib/share/opencc/jyutjyu.json
!cp ./jyutjyu.ocd2 ./venv/lib/site-packages/opencc/clib/share/opencc/jyutjyu.ocd2

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
import opencc
t2s = opencc.OpenCC('t2s.json')
input_text = t2s.convert("港珠澳大橋金巴往澳門須排至少2小時")

stn_tst = get_text("[GD]" + input_text + "[GD]", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1.2)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))